# Mastering Applied Skills in Management, Analytics and Entrepreneurship

## DATA COLLECTION TECHNIQUES
## Part I. Connect to remote database

__NOTE:__ use this notebook with `Data Science environment`.

### 1. Connect to PostgreSQL database

In [ ]:
import os
import json
import psycopg2

[Psycopg](https://www.psycopg.org/docs/index.html) is already installed in the JupyterHub environment as a PostgreSQL database adapter for Python.

#### 1.1. Bad practice

In [ ]:
# do not do like this!
# it is not safe way
# to store your credentials
POSTGRESQL_HOST = 'myhost'
DBNAME = 'mydbase'
USERNAME = 'mynamr'
PASSWORD = 'youshallnotpass'

#### 1.2. Good practice

The best solution __for production__ is to use environment variables. About [environment variables](https://help.ubuntu.com/community/EnvironmentVariables) and [how to use it](https://www.twilio.com/blog/environment-variables-python).

In [ ]:
# list all environmental variables
os.environ

In [ ]:
# good practice is to name environment variables
# with use of CAPITAL letters
os.environ['POSTGRESQL_HOST'] = 'your_host'
os.environ['DBNAME'] = 'your_dbname'
os.environ['USERNAME'] = 'your_username'
os.environ['PASSWORD'] = 'your_password'

In [ ]:
os.environ['POSTGRESQL_HOST']

...but again the code will contain sensitive credentials data, so let's use a small trick.

#### 1.3. A few words about dictionaries ans JSON

[JSON](https://docs.python.org/3/library/json.html) is cool!

In [ ]:
{'key': 'value', 'one_more_key': 'and the other value'}

In [ ]:
d = {'key': 'value', 'one_more_key': 'and the other value'}
type(d)

In [ ]:
d['key']

In [ ]:
json.dumps(d)

In [ ]:
with open('data/file_with_d.json', 'w') as file:
    json.dump(d, file)

In [ ]:
with open('data/file_with_d.json') as file:
    new_d = json.load(file)

In [ ]:
type(new_d)

In [ ]:
new_d

#### 1.4. Our solution to store credentiials

Create a file with the credentials and keep it safe from direct use of its content within the code.

In [ ]:
os.listdir()

In [ ]:
# this way is better
def access_data(file_path):
    """
    Reads JSON data from file.
    JSON data is a dictionary
    in Python.

    Keyword agruments:
      file_path: path to a file with JSON data

    By the way, here is a guide for docstring
    for Python functions:
    https://peps.python.org/pep-0257/

    """
    with open(file_path) as file:
        access_data = json.load(file)
    return access_data


creds = access_data(file_path='data/access_dbase.json')
print(creds.keys())

In [ ]:
conn = psycopg2.connect(
    dbname=creds['DBNAME'],
    user=creds['USERNAME'],
    password=creds['PASSWORD'],
    host=creds['POSTGRESQL_HOST'],
    port='6432'
)
cur = conn.cursor()

### 2. How to send query

In [ ]:
query = 'SELECT * FROM pg_catalog.pg_tables;'

In [ ]:
cur.execute(query)
records = cur.fetchall()
cur.close()
conn.close()

In [ ]:
del conn, cur

In [ ]:
print(records[0])

### 3. How to send query ('with' context manager)

The [with](https://docs.python.org/3/reference/compound_stmts.html#with) statement is used to wrap the execution of a block with methods defined by a context manager (see section [With Statement Context Managers](https://docs.python.org/3/reference/datamodel.html#context-managers)). 

In [ ]:
with psycopg2.connect(
    dbname=creds['DBNAME'], 
    user=creds['USERNAME'],
    password=creds['PASSWORD'], 
    host=creds['POSTGRESQL_HOST'],
    port='6432'
) as conn:
    with conn.cursor() as cur:
        cur.execute(query)
        records = cur.fetchall()

In [ ]:
print(records[0])

In [ ]:
# NOTE that they are closed after use becuase of `with`
conn, cur

### 4. Multiply queries

In [ ]:
# here are queries to database
# first is for all names of databases
# second is for all names of tables
# third is for all columns in table `jhublogs`
queries = {
    '___DATABASES___': 'SELECT * FROM pg_database',
    '___TABLES___': "SELECT relname FROM pg_class WHERE relkind='r' AND relname !~ '^(pg_|sql_)';",
    '___COLUMNS___': "SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'jhublogs';"
}

In [ ]:
with psycopg2.connect(
    dbname=creds['DBNAME'], 
    user=creds['USERNAME'],
    password=creds['PASSWORD'], 
    host=creds['POSTGRESQL_HOST'],
    port='6432'
) as conn:
    # simple loop for many queries
    for name, query in queries.items():
        print('\n', name)
        with conn.cursor() as cur:
            cur.execute(query)
            for row in cur.fetchall():
                print(row)

### 5. SQL for IPython

The [IPython SQL](https://github.com/catherinedevlin/ipython-sql) framework allows to use [magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html) like `%sql` (or `%%sql`) for SQL queries. We can connect to a database, then issue SQL commands within Jupyter.

#### 5.1. Install IPython SQL

In [ ]:
!pip install ipython-sql

In [ ]:
%load_ext sql

Connection data string to be used later:

#### 5.2. HINT to work with strings in Python

How to insert variables into string? There are many options but will look at two of them:

##### Option 1: f-string approach

In [ ]:
# f-string approach
var = 'Bob'
my_string = 'Hello, Bob!'
print(my_string)

In [ ]:
var = 'Bob'
my_string = f'Hello, {var}!'
print(my_string)

##### Option 2: .format() approach

In [ ]:
var1 = 'Bob'
var2 = 'Alice'

In [ ]:
my_string = 'Hello, {} and {}!'.format(var1, var2)
print(my_string)

#### 5.3. Let's connect!

In [ ]:
CONNECT_DATA = 'postgresql://{}:{}@{}:{}/{}'.format(
    creds['USERNAME'],
    creds['PASSWORD'], 
    creds['POSTGRESQL_HOST'],
    '6432',
    'dbsimba'
)
CONNECT_DATA

Let's look at all databases in PostgreSQL. SQL query can be done after the connection with `%%sql` magic command:

In [ ]:
%%sql $CONNECT_DATA
    SELECT * FROM public.jhublogs LIMIT 5

In [ ]:
%sql SELECT tablename AS table FROM pg_tables WHERE tablename !~ '^(pg_|sql_)'

## <font color='red'>INTERMEDIATE QUIZ #1-1</font>
You are exploring the data on __JupyterHub platform__ (https://jhas01.gsom.spbu.ru). Our database contains table of logs which is called `jhublogs` as you may have noticed.

Answer the following questions with the help of `%%sql`:
1. How many logs are there? (HINT: use `COUNT` command)
2. What time interval is covered by logs? (HINT: use `max` and `min` commands over `log_timestamp` column)
3. Find all types of logs (HINT: use `log_type` column and `DISTINCT` method)
4. Find all logs that are produced by your activity? (HINT: use `LIKE '%mylogin%'` syntax)

In [ ]:
### YOUR CODE HERE ###

### 6. How to deal with the data

We like [Pandas dataframes](https://pandas.pydata.org/) so we can convert our data from the database to that format.

In [ ]:
query = 'SELECT * FROM public.jhublogs LIMIT 100000;'

In [ ]:
with psycopg2.connect(
    dbname=creds['DBNAME'],
    user=creds['USERNAME'],
    password=creds['PASSWORD'],
    host=creds['POSTGRESQL_HOST'],
    port='6432'
) as conn:
    with conn.cursor() as cur:
        cur.execute(query)
        # names of table columns
        col_names = [desc[0] for desc in cur.description]
        # data from table
        records = cur.fetchall()

In [ ]:
print(records[:3])

In [ ]:
print(col_names)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(records, index=None)
df.columns = col_names
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

### 7. ClickHouse connection demo

[ClickHouse](https://clickhouse.com/) is the fastest and most resource efficient open-source database for real-time apps and analytics. 

How can we connect to it? We will need very famous [Requests](https://requests.readthedocs.io/en/latest/index.html) library.

In [ ]:
import requests

In [ ]:
CH_HOST = 'http://10.129.0.30'
CH_PORT = '8123'
SSL_VERIFY = True
query = 'SHOW DATABASES'
user_name = os.environ['CLICKHOUSE_USER']
user_passwd = os.environ['CLICKHOUSE_PASSWORD']

In [ ]:
r = requests.post(
    url=':'.join([CH_HOST, CH_PORT]),
    data=query,
    auth=(
        os.environ['CLICKHOUSE_USER'],
        os.environ['CLICKHOUSE_PASSWORD']
    ),
    verify=SSL_VERIFY
)
print('request status code:', r.status_code)

Some useful info about [HTTP response status codes](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status).

In [ ]:
r

In [ ]:
r.raw

In [ ]:
r.text

In [ ]:
def get_data(query, host, user_name, user_passwd):
    """
    This function takes connection arguments
    and gets back the data from database
    we connect to.

    Keyword agruments:
      query: is SQL like query
      host: database host
      user_name: name of user
      user_passwd: is a password

    """
    if (user_name == '') and (user_passwd == ''):
        r = requests.post(
            host, 
            params=query_dict, 
            verify=SSL_VERIFY
        )
    else:
        r = requests.post(
            host, 
            data=query,
            auth=(
                user_name, 
                user_passwd
            ), 
            verify=SSL_VERIFY
        )
    print('request status code:', r.status_code)
    return r.text

In [ ]:
query = 'SHOW TABLES FROM gsomlogs'
data = get_data(
    query=query,
    host=':'.join([CH_HOST, CH_PORT]),
    user_name=os.environ['CLICKHOUSE_USER'],
    user_passwd=os.environ['CLICKHOUSE_PASSWORD']
)

In [ ]:
# raw format
data

In [ ]:
data = [x.split('\t') for x in data.split('\n')]
pd.DataFrame(data)

In [ ]:
query = 'SELECT * FROM gsomlogs.hits_all ORDER BY DateTime DESC LIMIT 100'
data = get_data(
    query=query,
    host=':'.join([CH_HOST, CH_PORT]),
    user_name=os.environ['CLICKHOUSE_USER'],
    user_passwd=os.environ['CLICKHOUSE_PASSWORD']
)

In [ ]:
data = [x.split('\t') for x in data.split('\n')]
df = pd.DataFrame(data)
df.head()

## <font color='red'>LAB WORK #1</font>

Your lab work is as follows:
1. Take Jupyter logs from database (PostgreSQL)
2. Explore them 
3. Find users that log in Jupyter most often

__HINT:__ look through fields `log_code` or `log_type` to undercover log in activities

In [ ]:
### YOUR CODE HERE ###